# --- Day 7: Camel Cards ---
https://adventofcode.com/2023/day/7

In [2]:
def getHands():
    with open("hands.txt") as file:
        return file.read()

In [3]:
#formatting
hands = getHands().split("\n")
hands = [x.split() for x in hands]
cards = [x[0] for x in hands] #Just keeps track of the cards (letters)
bids = [int(x[1]) for x in hands] #Just keeps track of the bids
handDict = dict(zip(cards, bids))

#Keeps track of card values
cardValues = dict({
    "A":14,
    "K":13,
    "Q":12,
    "J":11,
    "T":10,
    "9":9,
    "8":8,
    "7":7,
    "6":6,
    "5":5,
    "4":4,
    "3":3,
    "2":2
})

#Hand rankings (0:high card, 1:one pair, 2:two pair, 3:three of a kind, 4:full house, 5:four of a kind, 6:five of a kind)
handTypes = [[], [], [], [], [], [], []]

#This sorts each 
for hand in cards:
    #Check for high card
    if len(set(hand)) == 5:
        handTypes[0].append(hand)
        
    #Checks for one pair
    elif len(set(hand)) == 4:
        handTypes[1].append(hand)
        
    #Checks for two pair and three of a kind
    elif len(set(hand)) == 3:
        
        #Counts the occurences of each card in hand
        cardCount = dict()
        for card in hand:
            if card not in cardCount.keys():
                cardCount[card] = 1
            else:
                cardCount[card] += 1
                
        #If a card does not occur 3 times then it is a two pair
        if 3 not in cardCount.values():
            handTypes[2].append(hand)
            continue
        #Otherwise it is a three of a kind
        handTypes[3].append(hand)
        
    #Checks for full house and four of a kind
    elif len(set(hand)) == 2:
        
        #Counts the occurences of each card in hand
        cardCount = dict()
        for card in hand:
            if card not in cardCount.keys():
                cardCount[card] = 1
            else:
                cardCount[card] += 1
                
        #If there are not 4 of one card type in the hand, then it's a full house
        if 4 not in cardCount.values():
            handTypes[4].append(hand)
            continue
        #Otherwise it's a four of a kind
        handTypes[5].append(hand)
        
    #Checks for five of a kind
    elif len(set(hand)) == 1:
        handTypes[6].append(hand)
        
#Gets a list of all hands in order
rankedHands = []
for i in handTypes:
    #For every hand type, add all hands to ranked hands after sorting them by their card value
    rankedHands.extend(sorted(i, key=lambda cards: [cardValues[x] for x in cards]))
    
#Get the total winnings by multiplying the bet and rank for each hand in rankedHands and getting the sum
totalWinnings = sum([handDict[hand] * rank for rank, hand in enumerate(rankedHands, 1)])
print(f"Total winnings: {totalWinnings}")

Total winnings: 250474325


## Slightly different part 1 solution

In [5]:
#Formatting
hands = getHands().split("\n")
cards = [hand.split()[0] for hand in hands]
bets = [int(hand.split()[1]) for hand in hands]
cardBets = dict(zip(cards, bets))

#Sort cards based on letter ordering
cardsOrder = "23456789TJQKA"
cards = sorted(cards, key=lambda card: [cardsOrder.find(x) for x in card]) #Sort by the index of cardsOrder

#Keep track of hand types
#0: high card, 1: one pair, 2: two pair, 3: three of a kind, 4: full house, 5: four of a kind, 6: five of a kind
handTypes = [[], [], [], [], [], [], []]

#Loop through each hand in cards
for hand in cards:
    
    #Checks for high card hand
    if len(set(hand)) == 5:
        handTypes[0].append(hand)
        continue
        
    #Checks for one pair
    if len(set(hand)) == 4:
        handTypes[1].append(hand)
        continue
        
    #Checks for two pair and three of a kind
    if len(set(hand)) == 3:
        #Check for three of a kind
        #If none of the first three cards have 3 occurrences in the hand, it is not three of a kind
        if hand.count(hand[0]) == 3 or hand.count(hand[1]) == 3 or hand.count(hand[2]) == 3:
            handTypes[3].append(hand)
            continue
        #Otherwise it is a two pair
        handTypes[2].append(hand)
        continue
    
    #Checks for full house and four of a kind
    if len(set(hand)) == 2:
        #Check for four of a kind
        #If either of the first two cards in a hand appear 4 times, then it is a four of a kind
        if hand.count(hand[0]) == 4 or hand.count(hand[1]) == 4:
            handTypes[5].append(hand)
            continue
        #Otherwise it's a full house
        handTypes[4].append(hand)
        continue
        
    #Checks for five of a kind
    if len(set(hand)) == 1:
        handTypes[6].append(hand)
        continue
        
#Flatten handTypes list as a new list: rankedHands
rankedHands = []
for i in handTypes:
    rankedHands.extend(i)

#Calculate total winnings
totalWinnings = 0
for rank, hand in enumerate(rankedHands, 1):
    totalWinnings += rank*cardBets[hand]
    
print(f"Total winnings: {totalWinnings}")

Total winnings: 250474325


# --- Part Two ---

In [8]:
import itertools

def getBestHandType(hand):
    """
    This function takes in a string: hand (5 characters representing each card)
    If there is a joker in the hand, it will get every combination that the hand can be with said jokers
    It goes through, and finds the highest ranking hand that the hand can be (if no jokers, then just the hand)
    Return: The highest ranking hand it can be
    """
    
    numJokers = hand.count("J")
    
    #If there are jokers in the hand
    if numJokers:
    
        #Check for known cases to avoid unnecessary work
        #Checks for 5 of a kind
        if numJokers == 5 or numJokers == 4:
            return "Five of a kind"
        #If there are 3 jokers and the other two cards are the same, then it's a 5 of a kind, otherwise, 4 of a kind
        if numJokers == 3 and hand.replace("J", "")[0] == hand.replace("J", "")[1]:
            return "Five of a kind"
        elif numJokers == 3:
            return "Four of a kind"

        #Get all possible hands with jokers
        hands = []
        #Loop through all combinations of joker replacements:
        for combos in itertools.combinations_with_replacement("23456789TQKA", numJokers):
            #Reset current hand so we can make a new hand with each combination
            currentHand = hand

            #Loop through each replacement in the combination
            for replacement in combos:
                #Replace the next "J" with the replacement
                currentHand = currentHand.replace("J", replacement, 1)
            #After all "J"s have been replaced, add it to the list of all hands
            hands.append(currentHand)
    
    #If there are no jokers in the hand, set hands = a list with just the original hand
    else:
        hands = [hand]
        
    
    #Get all hand types, and then return the highest valued type
    handTypes = []
    #Loop through hands:
    for hand in hands:
        
        #Checks for high card hand
        if len(set(hand)) == 5:
            handTypes.append("High card")
            continue
        
        #Checks for one pair
        if len(set(hand)) == 4:
            handTypes.append("One pair")
            continue
        
        #Checks for two pair and three of a kind
        if len(set(hand)) == 3:
            #Check for three of a kind
            #If none of the first three cards have 3 occurrences in the hand, it is not three of a kind
            if hand.count(hand[0]) == 3 or hand.count(hand[1]) == 3 or hand.count(hand[2]) == 3:
                handTypes.append("Three of a kind")
                continue
            #Otherwise it is a two pair
            handTypes.append("Two pair")
            continue
            
        #Check for full house and four of a kind
        if len(set(hand)) == 2:
            #Check for four of a kind
            #If either of the first two cards in a hand appear 4 times, then it is a four of a kind
            if hand.count(hand[0]) == 4 or hand.count(hand[1]) == 4:
                handTypes.append("Four of a kind")
                continue
            #Otherwise it is a full house
            handTypes.append("Full house")
            continue
            
        #If there is only 1 unique card, it is a five of a kind
        if len(set(hand)) == 1:
            return "Five of a kind"
    
    
    #Return the highest hand type in handTypes
    handTypeOrder = ["Five of a kind","Four of a kind","Full house","Three of a kind","Two pair","One pair","High card"]
    for i in handTypeOrder:
        if i in handTypes:
            return i
       
    
#Formatting
hands = getHands().split("\n")
cards = [x.split()[0] for x in hands]
bets = [int(x.split()[1]) for x in hands]
cardBets = dict(zip(cards, bets))

#Sort hands by custom card ranking
cardRanking = "J23456789TQKA"
cards = sorted(cards, key=lambda cards: [cardRanking.find(char) for char in cards])

#Keep track of hand types
handTypeKey={"High card": 0, "One pair": 1, "Two pair": 2, "Three of a kind": 3, "Full house": 4,
             "Four of a kind": 5, "Five of a kind": 6}
handTypes = [[], [], [], [], [], [], []]

#Loop through all cards
for hand in cards:
    #Get the best hand type
    bestHandType = getBestHandType(hand)
    #Append the hand to the list associated with the correct hand type
    handTypes[handTypeKey[bestHandType]].append(hand)

#Flatten handTypes into new list: rankedHands
rankedHands = []
for handType in handTypes:
    rankedHands.extend(handType)
    
#Calculate total winnings
totalWinnings = 0
for rank, hand in enumerate(rankedHands, 1):
    totalWinnings += rank * cardBets[hand]
    
print(f"Total winnings: {totalWinnings}")

Total winnings: 248909434


In [7]:
#Master plan:

#Sort by alphabet
#Sort into handTypes
    #If joker in hand: get best hand type for that hand
    #Otherwise, sort into lists regularly
#Flatten list
#Calculate score
#You are awesome!